In [1]:
# PATH = '/Users/ivan_zorin/Documents/DEV/code/VALL-E-X/' # MAC
PATH = '/home/ivan/dev/code/VALL-E-X/' # SK DESKTOP
# PATH = '/beegfs/home/ivan.zorin/dev/code/VALL-E-X/' # ZHORES

In [31]:
import sys
sys.path.append(PATH)

from utils.generation import SAMPLE_RATE, generate_audio, preload_models
from scipy.io.wavfile import write as write_wav
from IPython.display import Audio, display

import os
import numpy as np
import scipy
import soundfile as sf

import torch

In [3]:
# download and load all models
preload_models()

/home/ivan/opt/miniconda3/envs/tts/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [13]:
# init quality predictor
predictor = torch.hub.load("tarepan/SpeechMOS:v1.2.0", "utmos22_strong", trust_repo=True)

Using cache found in /home/ivan/.cache/torch/hub/tarepan_SpeechMOS_v1.2.0
/home/ivan/opt/miniconda3/envs/tts/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [4]:
presets_path = os.path.join(PATH, 'presets')
presets = os.listdir(presets_path)
print(presets)

['neutral.npz', 'librispeech_1.npz', 'amused.npz', 'paimon.npz', 'zh2en_tts_2.npz', 'alan.npz', 'acou_4.npz', 'vctk_2.npz', 'sleepiness.npz', 'emo_sleepy.npz', 'acou_1.npz', 'zh2en_tts_1.npz', 'dingzhen.npz', 'acou_2.npz', 'en2zh_tts_1.npz', 'en2zh_tts_3.npz', 'zh2en_tts_4.npz', 'seel.npz', 'babara.npz', 'emo_amused.npz', 'anger.npz', 'disgust.npz', 'vctk_3.npz', 'en2zh_tts_2.npz', 'librispeech_2.npz', 'rosalia.npz', 'esta.npz', 'en2zh_tts_4.npz', 'emo_anger.npz', 'fuxuan.npz', 'librispeech_4.npz', 'bronya.npz', 'cafe.npz', 'yaesakura.npz', 'emo_neutral.npz', 'vctk_1.npz', 'librispeech_3.npz', 'emotion_sleepiness.npz', 'vctk_4.npz', 'acou_3.npz', 'zh2en_tts_3.npz']


In [46]:
def generation_try():
    preset = presets[0]
    prompt = presets_path + '/' + preset
    audio_array = generate_audio(text_prompt, prompt=prompt)
    print(preset)
    display(Audio(audio_array, rate=SAMPLE_RATE))
    return audio_array

def mos_scoring(wavs, predictor=None, sr=SAMPLE_RATE):
    if predictor is None:
        predictor = torch.hub.load("tarepan/SpeechMOS:v1.2.0", "utmos22_strong", trust_repo=True)
    
    scores = []
    for wav in wavs:
        score = predictor(torch.tensor(wav[None], dtype=torch.float32), sr=sr)
        scores.append(score)
    return scores

In [49]:
# generate audio from text
texts = [
    """
    Hello, my name is Adolf. Recently I moved to Makhachkala, Dagestan Republic. Among many of my interests is to visit airports
    """,
    """
    THE ELECTRIC LIGHT CAN SCARCELY PENETRATE THROUGH THE DENSE CURTAIN WHICH HAS DROPPED OVER THE THEATRE ON WHICH THE BATTLE OF THE ELEMENTS IS ABOUT TO BE WAGED
    """   
]

idx = 1
text_prompt = texts[idx]


In [ ]:
preset = presets[0]
prompt = presets_path + '/' + preset
audio_array = generate_audio(text_prompt, prompt=prompt)
print(preset)
display(Audio(audio_array, rate=SAMPLE_RATE))

In [10]:
wav1 = generation_try()
wav2 = generation_try()
wav3 = generation_try()


VALL-E EOS [308 -> 1024]
neutral.npz


VALL-E EOS [308 -> 1117]
neutral.npz


VALL-E EOS [308 -> 952]
neutral.npz


In [32]:
real_speech_path = "/home/ivan/dev/data/librispeech/test-clean/test-clean/260/123288/260-123288-0003.flac"
real_speech_wav, real_speech_sr = sf.read(real_speech_path)

In [26]:
scores = mos_scoring([wav1, wav2, wav3], predictor=predictor, sr=SAMPLE_RATE)

In [47]:
real_speech_score = mos_scoring([real_speech_wav], predictor=predictor, sr=real_speech_sr)

In [48]:
print(scores)
print(real_speech_score)

[tensor([3.6633], grad_fn=<AddBackward0>), tensor([3.6008], grad_fn=<AddBackward0>), tensor([3.9430], grad_fn=<AddBackward0>)]
[tensor([4.4096], grad_fn=<AddBackward0>)]


In [ ]:

for preset in presets:
    prompt = presets_path + preset
    audio_array = generate_audio(text_prompt, prompt=prompt)
    print(preset)
    display(Audio(audio_array, rate=SAMPLE_RATE))
    print('='*10)

In [ ]:
from utils.prompt_making import make_prompt

audio_prompt_path = '/Users/ivan_zorin/Documents/DEV/code/tts/me_eng.wav'
### Use given transcript
make_prompt(name="ivan", audio_prompt_path=audio_prompt_path,
                transcript="Hello, my name is Ivan. One, two, three. This is my voice prompt. One more sentence to meet the time requirement.")

### Alternatively, use whisper
# make_prompt(name="ivan", audio_prompt_path=audio_prompt_path)

In [ ]:
text = """
Hello, my name is Ivan. One, two, three. This is my voice prompt. One more sentence to meet the time requirement. The brown for jumped over the lazy dog.
"""

audio_array = generate_audio(text, prompt="ivan", language='en')
Audio(audio_array, rate=SAMPLE_RATE)


In [ ]:
# save audio to disk
# write_wav("vallex_generation.wav", SAMPLE_RATE, audio_array)

# play text in notebook
Audio(audio_array, rate=SAMPLE_RATE)

### Coqui

In [ ]:
from TTS.api import TTS

In [ ]:
models = TTS().list_models()
models


In [ ]:
model_name = models[0]

tts = TTS(model_name)